In [1]:
!pip install resampy


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import os
import time
import joblib
import librosa
import numpy as np
from config import SAVE_DIR_PATH
from config import TRAINING_FILES_PATH
class CreateFeatures:
    @staticmethod
    def features_creator(path, save_dir) -> str:
        lst = []
        start_time = time.time()
        for subdir, dirs, files in os.walk(path):
            for file in files:
                try:
                    X, sample_rate = librosa.load(os.path.join(subdir, file),res_type='kaiser_fast')
                    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate,n_mfcc=40).T, axis=0)
                    file = int(file[7:8]) - 1
                    arr = mfccs, file
                    lst.append(arr)
                except ValueError as err:
                    print(err)
                    continue
        print("--- Data loaded. Loading time: %s seconds ---" % (time.time() - start_time))
        X, y = zip(*lst)
        X, y = np.asarray(X), np.asarray(y)
        print(X.shape, y.shape)
        X_name, y_name = 'X.joblib', 'y.joblib'
        joblib.dump(X, os.path.join(save_dir, X_name))
        joblib.dump(y, os.path.join(save_dir, y_name))
        return "Completed"
if __name__ == '__main__':
    print('Routine started')
    FEATURES = CreateFeatures.features_creator(path=TRAINING_FILES_PATH, save_dir=SAVE_DIR_PATH)
    print('Routine completed.')

Routine started
--- Data loaded. Loading time: 373.15474581718445 seconds ---
(5252, 40) (5252,)
Routine completed.
